# Challenge 2 Critique de Films

https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,asc&view=advanced

## Structure de l'URL

### Training

In [1]:
# conda install -c anaconda beautifulsoup4 à faire avant dans la console
import requests
# Librairie BeautifulSoup du package bs4
from bs4 import BeautifulSoup

In [10]:
#Solution Bruno
# Télécharger la page
url = "https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,desc"
response = requests.get(url)
content = response.content
parser = BeautifulSoup(content, 'html.parser')

print(parser.text[:5])

In [12]:
#Solution Rod
# Télécharger la page
from requests import get
url = 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,desc&ref_=adv_prv'
response = requests.get(url)
print(response.text[:100])



<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/
